In [96]:
import numpy as np
class board:
    def __init__(self,obsticles,m=18,n=9):
        self.m=m
        self.n=n
        self.pg=[]
        self.obsticles=obsticles
        self.dots=np.ones([m,n])
    def copy(self):
        e=board(self.obsticles.copy(),self.m,self.n)
        e.pg=self.pg.copy()
        e.dots=self.dots.copy()
        return e
    def utility(self):
        return 5
    def add_pg(self,pg):
        self.pg.append(pg)
    def request_move(self,w,index):
        pg=self.pg[index]
        next=np.array([pg.x,pg.y])
        if w==0  and pg.y<self.n-1:
            next=np.array([pg.x,pg.y+1])
        elif w==1  and pg.x<self.m-1:
            next=np.array([pg.x+1,pg.y])
        elif w==2  and pg.y>0:
            next=np.array([pg.x,pg.y-1])
        elif w==3 and pg.x>0 :
            next=np.array([pg.x-1,pg.y])
        if index==0 :
            pg.score=pg.score-1
            if self.dots[next[0],next[1]]==1:
                self.dots[next[0],next[1]]=0
                pg.score=pg.score+10
        if not((self.obsticles==next).all(axis=1)).any():
            pg.x,pg.y=next[0],next[1]
    def Map(self):
        b=np.zeros([self.m,self.n],dtype=np.int8)
        for i in self.pg:
            b[i.x,i.y]=2
        b[self.pg[0].x,self.pg[0].y]=1
        b[self.obsticles[:,0],self.obsticles[:,1]]=-1
        return b  
    def step(self):
        for i in self.pg:
             i.go()          
        

In [97]:
class PG:
    def __init__(self,x,y,enviroment):
        self.x=x
        self.y=y
        self.enviroment=enviroment
        enviroment.add_pg(self)
        self.index=len(enviroment.pg)-1
    def move(self,w):
        self.enviroment.request_move(w,self.index)


In [106]:
class Pacman(PG):
    max_depth=5
    def __init__(self, x, y, enviroment=None):
        super().__init__(x, y, enviroment)
        self.score=10
        self.enviroment.dots[x,y]=0
    def desision(self):
        b=self.enviroment.copy()
        max_i,i=self.Max_search(b)
        return i
    def Max_search(self,b,depth=0):
        l=[]
        if depth<Pacman.max_depth:
            for i in range(4):
                b.pg[0].move(i)
                l.append(b.pg[0].Min_search(b,depth+1))
                b.pg[0].move((i+2)%4)
            return max(l),np.argmax(l)
        else:
            return b.utility()
    def Min_search(self,b,depth):
        l=[]
        if depth<Pacman.max_depth:
            for j in range(1,5):
                b.pg[1].move(j)
                for k in range(1,5):
                    b.pg[2].move(k)
                    l.append(b.pg[0].Max_search(b,depth+1)[0])
                    b.pg[2].move((k+2)%4)
                b.pg[1].move((j+2)%4)
            return min(l)
        else:
            return b.utility()
    def go(self):
        i=self.desision()
        self.move(i)

In [99]:
class ghost(PG):
    def __init__(self, x, y, enviroment=None):
        super().__init__(x, y, enviroment)
    def go(self):
        self.move(np.random.choice(4))


In [100]:
p.desision()

0

In [107]:
m=18
n=9
k=4
obs=np.array([[1,2],[3,4]])
b=board(obs,m,n)
p=Pacman(5,5,b)
g1=ghost(7,7,b)
g2=ghost(6,6,b)


In [110]:
b.step()
b.Map()

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [125]:
def random_place(m,n,k):
    index_array1 = np.arange(m)
    index_array2 = np.arange(n)
    np.random.shuffle(index_array1)
    np.random.shuffle(index_array2)
    x,y=index_array1[0:k],index_array2[0:k]
    return x,y

In [95]:

def random_place(m,n,k):
    index_array1 = np.arange(m)
    index_array2 = np.arange(n)
    np.random.shuffle(index_array1)
    np.random.shuffle(index_array2)
    x,y=index_array1[0:k],index_array2[0:k]
    x=np.concatenate((x,y)).reshape((k,2))
   
    return x
    # def sucssesor(self,l):
    #     b=self.enviroment.copy()
    #     pg=b.pg
    #     for i in range(1,5):
    #         pg[0].move(i)
    #         for j in range(1,5):
    #             pg[1].move(j)
    #             for k in range(1,5):
    #                 pg[2].move(k)
    #                 l.append(b.Map())
    #                 pg[1].move((i+2)%4+1)
    #                 pg[2].move((i+2)%4+1)
    #         pg[0].move((i+2)%4+1)

0

In [112]:
class a():
    s=15
    def __init__(self,x) -> None:
        self.x=x
    def f(self,r):
        if a.s>r:
            print('hello')
b=a(5)
